# Processing Issues using Gemma7b

In [4]:
import ollama
import json
import tiktoken
from llama_index.core.node_parser import TokenTextSplitter
from functools import partial

In [52]:
with open('data/prediction_data/issues.json') as f:
    issues_data = [json.loads(line) for line in f]
#issues_data

### Text Processor

In [53]:
chunk_size = 6000
separator = '\n'
backup_separators = [".", " "]
tokenizer = partial(tiktoken.get_encoding("cl100k_base").encode, allowed_special="all")

def txt_splitter(text):
    text_splitter = TokenTextSplitter(separator=separator,
                                        chunk_size=chunk_size,
                                        backup_separators=backup_separators,
                                        tokenizer=tokenizer)
    # print(len(tokenizer(text))))  ### Check token size 
    if len(tokenizer(text)) <= chunk_size:
        return [text]
    
    chunks = text_splitter.split_text(text)
    return chunks

### Prepare Issues

In [54]:
for issues in issues_data:
    issues['issues'] = txt_splitter(issues['issues'])

## LLM Magic

Prompt Templates

In [69]:
def generate_summary_template(context, prev_summary=''):

    prompt_template = f"""
    You are a Singapore Lawyer. \n
    summarise the main issues of the legal judgment below in 1500 words:\n
    {prev_summary}\n
    {context}.
    """
    return prompt_template

def generate_final_template(context):
    prompt_template = f"""
    {context}.\n
    summarise the legal issues above into sentences separated by full-stop. DO NOT give any headers.

    EXAMPLE: (if there are 3 issues)
    Issue 1. Issue 2. Issue 3.
    """
    return prompt_template

In [68]:
for issues in issues_data:
    prev_summary = ''
    for chunk in issues['issues']:
        context = generate_summary_template(chunk, prev_summary)
        response = ollama.generate(model='gemma', prompt=str(context), stream=False)
        prev_summary = response['response']
    
    print(prev_summary)
    overall_summary = generate_final_template(prev_summary)
    response = ollama.generate(model='gemma', prompt=str(overall_summary), stream=False)
    final_summary = response['response']
    
    
    issues['summarised issues'] = final_summary.split('.')

## Main Issues of the Legal Judgment:

**1. Restriction on Multiple Actions:**

- The judgment clarifies that a plaintiff cannot be prohibited from suing a defendant solely because they failed to join them in previous proceedings involving another person.
- The court recognizes that such a failure can be used for cross-examination and cost implications, but suggests alternative measures before resorting to striking out the entire case.


**2. Abuse of Court Process:**

- The judgment examines the possibility that the plaintiff's action constitutes an abuse of court process.
- It finds that the plaintiff's claim of mistaken payment was not viable in the previous lawsuit due to the lack of knowledge of the mistake at the time.
- The court allows the plaintiff to rectify this oversight by bringing a fresh action.


**3. Discovery of Mistakes:**

- The judgment emphasizes the importance of discovering material errors in legal proceedings.
- If such mistakes are discovered after the initial

KeyboardInterrupt: 

In [67]:
for issues in issues_data:
    print(issues['summarised issues'])

["Striking out plaintiff's action, discovery of mistake of law, suability of the defendant, balancing interests,"]
['(You have not provided any legal issues for me to summarise', ')']
['Consistency with previous case law, applicability of Clause 8, award of costs', '']
['(Text not provided for summarisation', ')']
['The provided text does not include any legal issues or explanations, so I am unable to extract the requested data from the given context', '']
["Security for costs shall be refunded, applicants' solicitors shall be refunded", '']
['(Context not provided, so unable to extract legal issues', ')']
['(Text not provided for summarisation', ')']
['Natural justice concerns, jurisdiction issues, lack of notice and opportunity to be heard, and costs awarded', '']
['Right of Appeal, Forum for Appeal, Legislative Intent, Time Limit for Appeal', '']
['\n    ## Main Issues of the Legal Judgment:\n\n**1', ' Distinctiveness of the Figure "580":**\n\n- The judgment examines the question of

KeyError: 'summarised issues'